In [2]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

# Simulation Parameters
L = 10.0   # Length of the domain
T = 1.0    # Total simulation time
Nx = 100   # Number of spatial grid points
Nt = 201   # Number of time steps
dx = L / (Nx - 1)  # Spatial step size
dt = T / Nt        # Time step size

# Monte Carlo Simulations
for o in range(1000):
    D = np.random.uniform(1, 100)  # Diffusion coefficient
    R = abs(np.random.uniform(D - 10, D + 10)) % 100  # Sorption coefficient
    alpha = (D / R) * dt / dx**2  # Stability parameter

    print(f"Iteration {o}: D={D:.2f}, R={R:.2f}, alpha={alpha:.4f}")

    if alpha > 0.5:
        print("Skipping: alpha > 0.5 (unstable)")
        continue  # Skip this iteration if unstable

    # Initialize concentration array
    C = np.zeros(Nx)  
    C[int(Nx / 4):int(Nx / 2)] = 1.0  # Initial condition: step function

    # Boundary conditions (Dirichlet: fixed concentration)
    C[0] = 0.0
    C[-1] = 0.0

    # Store concentration data over time
    C_all = np.zeros((Nt, Nx))  

    # Time-stepping loop
    for t in range(Nt):
        C_new = C.copy()
        for i in range(1, Nx - 1):
            C_new[i] = C[i] + alpha * (C[i + 1] - 2 * C[i] + C[i - 1])
        C = C_new

        # Apply boundary conditions at each time step
        C[0] = 0.0
        C[-1] = 0.0

        # Store current concentration profile
        C_all[t, :] = C

    # Save results to HDF5 file
    try:
        with h5py.File("diff_sorp.h5", "a") as f:
            group_name = f"{o}"
            while group_name in f:
                group_name += "_retry"
            g = f.create_group(group_name)
            g.create_dataset("D", data=D)
            g.create_dataset("R", data=R)
            g.create_dataset("data", data=C_all)
    except Exception as e:
        print(f"Error saving iteration {o}: {e}")

print("Simulation completed.")


Iteration 0: D=48.45, R=54.71, alpha=0.4318
Iteration 1: D=58.85, R=61.99, alpha=0.4629
Iteration 2: D=11.24, R=8.57, alpha=0.6391
Skipping: alpha > 0.5 (unstable)
Iteration 3: D=10.29, R=19.61, alpha=0.2559
Iteration 4: D=9.27, R=6.63, alpha=0.6815
Skipping: alpha > 0.5 (unstable)
Iteration 5: D=11.81, R=9.23, alpha=0.6234
Skipping: alpha > 0.5 (unstable)
Iteration 6: D=1.15, R=6.08, alpha=0.0921
Iteration 7: D=68.07, R=69.13, alpha=0.4801
Iteration 8: D=95.25, R=3.55, alpha=13.0810
Skipping: alpha > 0.5 (unstable)
Iteration 9: D=53.70, R=55.08, alpha=0.4754
Iteration 10: D=67.00, R=72.14, alpha=0.4528
Iteration 11: D=16.62, R=23.93, alpha=0.3387
Iteration 12: D=89.51, R=87.97, alpha=0.4961
Iteration 13: D=95.73, R=5.70, alpha=8.1909
Skipping: alpha > 0.5 (unstable)
Iteration 14: D=47.39, R=38.75, alpha=0.5963
Skipping: alpha > 0.5 (unstable)
Iteration 15: D=41.56, R=49.77, alpha=0.4073
Iteration 16: D=11.36, R=20.23, alpha=0.2739
Iteration 17: D=89.64, R=80.66, alpha=0.5419
Skipping:

In [3]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import animation
from tqdm import tqdm
import h5py


def visualize_burgers(xcrd, data, path):
    """
    This function animates the Burgers equation

    Args:
    path : path to the desired file
    param: PDE parameter of the data shard to be visualized
    """
    fig, ax = plt.subplots()
    ims = []

    for i in tqdm(range(data.shape[0])):
        if i == 0:
            im = ax.plot(xcrd, data[i].squeeze(), animated=True, color="blue")
        else:
            im = ax.plot(xcrd, data[i].squeeze(), animated=True, color="blue")
        ims.append([im[0]])

    ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True, repeat_delay=1000)

    writer = animation.PillowWriter(fps=15, bitrate=1800)
    ani.save(path, writer=writer)
    plt.close(fig)

visualize_burgers([i for i in range(100)], C_all, "a.gif")

  0%|          | 0/201 [00:00<?, ?it/s]

100%|██████████| 201/201 [00:00<00:00, 6785.92it/s]


In [4]:
from scipy.stats import skewnorm

with h5py.File("diff_sorp.h5", "a") as f:
    for key in f.keys():
        u = f[key]["data"][:]
        noise = skewnorm.rvs(a=1, scale=0.2, size=u.shape)
        u_noise = u + noise
        f[key].create_dataset("noise", data=u_noise)